## LoL Champion Selection & Difficulty Analysis
----

This notebook uses heatmap visualizations to explore the relationship between champion archetypes and difficulty ratings in League of Legends. The analysis reveals patterns in how different champion types correlate with gameplay complexity, providing insights for new players when selecting champions while learning the game.

In [ ]:
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os

for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)

df = pd.read_csv(file_path)

: 

## Data Validation
----

Checking the data's shape along with any missing values.

In [2]:
df.shape

(172, 33)

In [3]:
df.head()

,Unnamed: 0,id,apiname,title,difficulty,herotype,alttype,resource,stats,rangetype,...,be,rp,skill_i,skill_q,skill_w,skill_e,skill_r,skills,fullname,nickname
0,Aatrox,266.0,Aatrox,the Darkin Blade,2,Fighter,Tank,Blood Well,"{'hp_base': 650, 'hp_lvl': 114, 'mp_base': 0, ...",Melee,...,4800,880,{1: 'Deathbringer Stance'},"{1: 'The Darkin Blade', 2: 'The Darkin Blade 3'}",{1: 'Infernal Chains'},{1: 'Umbral Dash'},{1: 'World Ender'},"{1: 'Deathbringer Stance', 2: 'The Darkin Blad...",NaN,NaN
1,Ahri,103.0,Ahri,the Nine-Tailed Fox,2,Mage,Assassin,Mana,"{'hp_base': 590, 'hp_lvl': 104, 'mp_base': 418...",Ranged,...,3150,790,{1: 'Essence Theft'},{1: 'Orb of Deception'},{1: 'Fox-Fire'},{1: 'Charm'},{1: 'Spirit Rush'},"{1: 'Essence Theft', 2: 'Orb of Deception', 3:...",NaN,NaN
2,Akali,84.0,Akali,the Rogue Assassin,2,Assassin,NaN,Energy,"{'hp_base': 600, 'hp_lvl': 119, 'mp_base': 200...",Melee,...,3150,790,"{1: ""Assassin's Mark""}",{1: 'Five Point Strike'},{1: 'Twilight Shroud'},{1: 'Shuriken Flip'},{1: 'Perfect Execution'},"{1: ""Assassin's Mark"", 2: 'Five Point Strike',...",Akali Jhomen Tethi,NaN
3,Akshan,166.0,Akshan,the Rogue Sentinel,3,Marksman,Assassin,Mana,"{'hp_base': 630, 'hp_lvl': 107, 'mp_base': 350...",Ranged,...,4800,880,{1: 'Dirty Fighting'},{1: 'Avengerang'},{1: 'Going Rogue'},{1: 'Heroic Swing'},{1: 'Comeuppance'},"{1: 'Dirty Fighting', 2: 'Avengerang', 3: 'Goi...",NaN,NaN
4,Alistar,12.0,Alistar,the Minotaur,1,Tank,Support,Mana,"{'hp_base': 685, 'hp_lvl': 120, 'mp_base': 350...",Melee,...,1350,585,{1: 'Triumphant Roar'},{1: 'Pulverize'},{1: 'Headbutt'},{1: 'Trample'},{1: 'Unbreakable Will'},"{1: 'Triumphant Roar', 2: 'Pulverize', 3: 'Hea...",NaN,NaN


In [4]:
total_nulls = df.isnull().sum().sum()
print(f'There are {total_nulls} nulls in the dataframe.')

There are 322 nulls in the dataframe.


In [5]:
df.dtypes

Unnamed: 0             object
id                    float64
apiname                object
title                  object
difficulty              int64
herotype               object
alttype                object
resource               object
stats                  object
rangetype              object
date                   object
patch                  object
changes                object
role                   object
client_positions       object
external_positions     object
damage                  int64
toughness               int64
control                 int64
mobility                int64
utility                 int64
style                   int64
adaptivetype           object
be                      int64
rp                      int64
skill_i                object
skill_q                object
skill_w                object
skill_e                object
skill_r                object
skills                 object
fullname               object
nickname               object
dtype: obj

In summary, the dataframe contains:
* 172 rows and 33 columns
* 322 nulls
* types: `object`, `float64`, `int64`

Based on the Data Card in Kaggle, the columns that are of interest are: `apiname`, `difficulty`, `herotype`,  `alttype`, `role`, `toughness`, `control`, `mobility`, `utility`, and `style`.

**Note:** Not all of these columns will be used when building out the heatmap. Instead they will serve as a reference for the categorical and numerical variables.

## Summary of Columns
----

* `apiname`: Champion's name within Riot's API
* `difficulty`: Champion's difficulty rating from 0 to 3
* `herotype`: Champion's archaic primary role
* `alltype`: Champion's archaic secondary role
* `role`: Champion's classes
* `toughness`: Champion's toughness rating from 1 to 3
* `control`: Champion's control rating from 1 to 3
* `mobility`: Champion's mobility rating from 1 to 3
* `utility`: Champion's utility rating from 1 to 3
* `style`: Champion's damage style (0 = attacker only; 100 = caster only)

## Creating the Heatmaps
----

In [6]:
# creating style categoories by binning the style score thresholds
df['style_category'] = pd.cut(df['style'], 
                             bins=[0, 25, 50, 75, 100], 
                             labels=['Attacker-Heavy', 'Attacker-Leaning', 'Balanced', 'Caster-Leaning'],
                             include_lowest=True)

# Champion Count by Hero Type and Difficulty Level
heatmap_data1 = df.groupby(['herotype', 'difficulty']).size().unstack(fill_value=0)

# hover text
hover_text = []
for herotype in heatmap_data1.index:
    hover_row = []
    for difficulty in heatmap_data1.columns:
        champions = df[(df['herotype'] == herotype) & (df['difficulty'] == difficulty)]['apiname'].tolist()
        champion_list = '<br>'.join(champions) if champions else 'No champions'
        hover_row.append(f"Hero Type: {herotype}<br>Difficulty: {difficulty}<br>Count: {heatmap_data1.loc[herotype, difficulty]}<br>Champions:<br>{champion_list}")
    hover_text.append(hover_row)

fig1 = go.Figure(data=go.Heatmap(
    z=heatmap_data1.values,
    x=heatmap_data1.columns,
    y=heatmap_data1.index,
    colorscale='YlOrRd',
    text=heatmap_data1.values,
    texttemplate="%{text}",
    textfont={"size": 12},
    hovertemplate='%{customdata}<extra></extra>',
    customdata=hover_text,
    colorbar=dict(title="Number of Champions")
))

fig1.update_layout(
    title="Champion Count by Hero Type and Difficulty Level<br><sub>Hover for champion details</sub>",
    xaxis_title="Difficulty Level",
    yaxis_title="Hero Type",
    width=800,
    height=600
)

fig1.show()

In [7]:
# Average Style Score by Hero Type and Difficulty
pivot_style = df.pivot_table(values='style', 
                            index='herotype', 
                            columns='difficulty', 
                            aggfunc='mean')

hover_text_style = []
for herotype in pivot_style.index:
    hover_row = []
    for difficulty in pivot_style.columns:
        champions_info = df[(df['herotype'] == herotype) & (df['difficulty'] == difficulty)]
        if not champions_info.empty:
            avg_style = champions_info['style'].mean()
            champion_details = []
            for _, champ in champions_info.iterrows():
                champion_details.append(f"{champ['apiname']}: {champ['style']}")
            detail_text = '<br>'.join(champion_details)
            hover_text_style_cell = f"Hero Type: {herotype}<br>Difficulty: {difficulty}<br>Avg Style: {avg_style:.1f}<br>Champions & Style Scores:<br>{detail_text}"
        else:
            hover_text_style_cell = f"Hero Type: {herotype}<br>Difficulty: {difficulty}<br>No data"
        hover_row.append(hover_text_style_cell)
    hover_text_style.append(hover_row)

fig2 = go.Figure(data=go.Heatmap(
    z=pivot_style.values,
    x=pivot_style.columns,
    y=pivot_style.index,
    colorscale='RdBu_r',
    zmid=50,
    text=np.round(pivot_style.values, 1),
    texttemplate="%{text}",
    textfont={"size": 10},
    hovertemplate='%{customdata}<extra></extra>',
    customdata=hover_text_style,
    colorbar=dict(title="Average Style Score<br>(0=Attacker, 100=Caster)")
))

fig2.update_layout(
    title="Average Style Score by Hero Type and Difficulty<br><sub>Hover for individual champion style scores</sub>",
    xaxis_title="Difficulty Level",
    yaxis_title="Hero Type",
    width=800,
    height=600
)

fig2.show()

In [8]:
# Style Category vs Difficulty
heatmap_data2 = df.groupby(['style_category', 'difficulty'], observed=False).size().unstack(fill_value=0)

hover_text_cat = []
for style_cat in heatmap_data2.index:
    hover_row = []
    for difficulty in heatmap_data2.columns:
        champions = df[(df['style_category'] == style_cat) & (df['difficulty'] == difficulty)]['apiname'].tolist()
        count = len(champions)
        
        if count == 0:
            champion_display = 'No champions'
        elif count <= 15:  # show ALL champs if 15 or fewer
            champion_display = '<br>'.join(champions)
        else:  # otherwise, show first 12 + remaining amount
            displayed_champions = champions[:12]
            remaining = count - 12
            champion_display = '<br>'.join(displayed_champions) + f'<br>... and {remaining} more'
        
        hover_row.append(f"Style Category: {style_cat}<br>Difficulty: {difficulty}<br>Count: {count}<br>Champions:<br>{champion_display}")
    hover_text_cat.append(hover_row)

fig3 = go.Figure(data=go.Heatmap(
    z=heatmap_data2.values,
    x=heatmap_data2.columns,
    y=heatmap_data2.index,
    colorscale='Viridis',
    text=heatmap_data2.values,
    texttemplate="%{text}",
    textfont={"size": 12},
    hovertemplate='%{customdata}<extra></extra>',
    customdata=hover_text_cat,
    colorbar=dict(title="Number of Champions")
))

fig3.update_layout(
    title="Champion Count by Style Category and Difficulty<br><sub>Hover for champion details</sub>",
    xaxis_title="Difficulty Level",
    yaxis_title="Style Category",
    width=800,
    height=500
)

fig3.show()

## Key Takeaways
----

Based on the `/kaggle/input/25-s1-3-league-of-legends-champion-data-2025` data, `Caster-Leaning` champions on average, are rated for a higher difficulty. When looking at the heatmap for `Champion Count by Style Category and Difficulty` there are a total of **37 champions rated between 1.5 - 2.5** in terms of difficulty.

The first heatmap `Champion Count by Hero Type and Difficulty Level` indeed shows that there are more champions with the `Mage` type classification between the ranges compared to Tanks, Supports, Marksmen, and Assassins. **The caveat here being that `Fighters` tend to play in the roles of top lane and jungle.**

**Caster-Leaning Champions include:**
* Ahri, Aurora, Brand, Braum, Fiddlesticks, Fizz, Galio, Gragas, Heimerdinger, Karthus, Kassadin, Katarina...and 25 more.

**The most difficult (rated 2.5 - 3.5) in the Caster-Leaning style category being Anivia, AurelionSol, Cassiopeia, Hwei, etc.**

These champions often utilize `"skill-shots"` that require the player to manually aim their champion's abilities in order to do damage. It is interesting to note that the champions rated with the most difficulty at least in the Caster-Leaning style category often go `Mid-lane` or `Mid` for short.

## Conclusion & Limitations
----

League of Legends offers numerous champions that fit different playstyles. The determination of whether a champion is less or more difficult is ultimately up to the player's experience with `MOBA: Multiplayer Online Battle Arena` games. In the case of League of Legends, `Caster-Leaning` champions are a great recommendation for intermediate-level players. 

***As a starting point for new players,*** the `Fighter` champions are rated lower in difficulty and are traditionally placed in roles such as `Top-lane` or `Jungle`. These roles often play against a single opponent or "camps" to obtain gold, experience, etc.

The limitations of this analysis include but are not limited to:
1. The dataset utilizes Riot's rating to assess champion difficulty rather than the player.
2. Champions that are less or more difficult wildly differ in elo-divisions. For example, a champion may be pereceived as lower difficulty in low-elo ennvironments. In high-elo environments, the same champion may have stricter execution requirements ultimately leading to a higher difficulty. This can be seen on websites that track overall winrates for champions based on the elo that they are being played at.
3. Factors that contribute to difficulty rating. Is it the total number of skillshots? Having a smaller or larger health pool? Having a smaller or larger mana pool? Damage numbers across various skill levels? Itemization costs and opportunities? Number of known champion counters? The list continues...
4. Changes in champion balancing and/or the introduction of new champions.